https://www.linkedin.com/learning/search?trk=homepage-basic_intent-module-learning&sortBy=RELEVANCE&entityType=COURSE

In [9]:
# importing the libraries
import requests # to allow http requests
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import os
import urllib.request as request

In [10]:
# begin with the url of the list of courses
url = 'https://www.linkedin.com/learning/search?trk=homepage-basic_intent-module-learning&sortBy=RELEVANCE&entityType=COURSE'


#ssr配置代理
# proxies={'http': 'http://127.0.0.1:4780', 'https': 'http://127.0.0.1:4780'}
url='https://www.linkedin.com/learning/search?trk=homepage-basic_intent-module-learning&sortBy=RELEVANCE&entityType=COURSE'
r = requests.get(url)

# r = requests.get(url,proxies=proxies)

soup = BeautifulSoup(r.text)
courses = soup.find_all('li',{'class':'results-list__item'})

In [18]:
courses[0]

<li class="results-list__item">
<a aria-label="The Extended Mind: Thinking outside the Brain (Book Bite), 14m, By: Next Big Idea Club, " class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link" data-tracking-control-name="learning-serp_learning-search-card_search-card" data-tracking-will-navigate="" href="https://www.linkedin.com/learning/the-extended-mind-thinking-outside-the-brain-book-bite?trk=learning-serp_learning-search-card_search-card" target="_self">
<!-- -->
<div class="search-entity-media search-entity-media--rectangle">
<img alt="The Extended Mind: Thinking outside the Brain (Book Bite)" class="search-entity-media__img" data-delayed-url="https://media-exp1.licdn.com/dms/image/C560DAQFIPuLe_EyZPw/learning-public-crop_288_512/0/1654818675296?e=1656000000&amp;v=beta&amp;t=i0xTv75QJgtthU-u2ONQ4PaYCDjo-c7aYrGcx132uW0"/>
<div class="search-entity-media__duration">
            
    
    
    
    
    
    
   

In [ ]:
urlList = []
durationList = []
nameList = []
byList = []
viewCountList = []
releaseDateList = []


for course in courses:
    url = course.find(href=True)
    print(url)
    urlList.append(url['href'])
    name = course.find('h3',{'class':'base-search-card__title'}).text.strip()
    nameList.append(name)
    by = course.find('h4',{'class':'base-search-card__subtitle'}).text.strip()
    byList.append(by)
    duration = course.find('div',{'class':'search-entity-media__duration'}).text.strip()
    durationList.append(duration)
    metadataItem = course.find_all('span')
    if "Released" in metadataItem[0].text:
        viewCountList.append(0)
        releaseDateList.append(metadataItem[0].text)
    else:
        viewCountList.append(metadataItem[0].text)
        releaseDateList.append(metadataItem[1].text)

[[<span class="base-search-card__metadata-item">Released Jun 1, 2022</span>],
 [<span class="base-search-card__metadata-item">Released Jun 1, 2022</span>],
 [<span class="base-search-card__metadata-item">Released Jun 1, 2022</span>],
 [<span class="base-search-card__metadata-item">Released Jun 1, 2022</span>],
 [<span class="base-search-card__metadata-item">Released Jun 1, 2022</span>],
 [<span class="base-search-card__metadata-item">Released Jun 2, 2022</span>],
 [<span class="base-search-card__metadata-item">Released Jun 2, 2022</span>],
 [<span class="base-search-card__metadata-item">Released Jun 2, 2022</span>],
 [<span class="base-search-card__metadata-item">Released May 31, 2022</span>],
 [<span class="base-search-card__metadata-item">Released May 31, 2022</span>],
 [<span class="base-search-card__metadata-item">Released May 31, 2022</span>],
 [<span class="base-search-card__metadata-item">Released May 31, 2022</span>],
 [<span class="base-search-card__metadata-item">Released May 31, 2022</span>],
 [<span class="base-search-card__metadata-item">Released May 31, 2022</span>],
 [<span class="base-search-card__metadata-item">Released May 31, 2022</span>],
 [<span class="base-search-card__metadata-item">Released May 31, 2022</span>],
 [<span class="base-search-card__metadata-item">Released Jun 1, 2022</span>],
 [<span class="base-search-card__metadata-item">181,477 viewers</span>,
  <span class="base-search-card__metadata-item">Released Dec 15, 2021</span>],
 [<span class="base-search-card__metadata-item">537,464 viewers</span>,
  <span class="base-search-card__metadata-item">Released Dec 15, 2021</span>],
 [<span class="base-search-card__metadata-item">1,071,484 viewers</span>,
  <span class="base-search-card__metadata-item">Released Sep 17, 2021</span>],

In [30]:
metadataItem

[<span class="base-search-card__metadata-item">1,414,881 viewers</span>,
 <span class="base-search-card__metadata-item">Released Nov 13, 2017</span>]

In [13]:
df = pd.DataFrame(list(zip(nameList,urlList,durationList,byList,viewCountList,releaseDateList)),
                 columns = ['course name','url','duration','by','viewer count','release date'])
df

,course name,url,duration,by,viewer count,release date
0,The Extended Mind: Thinking outside the Brain ...,https://www.linkedin.com/learning/the-extended...,14m,By: Next Big Idea Club,0,"Released Jun 15, 2022"
1,Branding with Creative Cloud Express,https://www.linkedin.com/learning/branding-wit...,1h 19m,By: Nicte Cuevas,0,"Released Jun 16, 2022"
2,Conducting Research in a Post-Pandemic World,https://www.linkedin.com/learning/conducting-r...,35m,By: Cory Lebson,0,"Released Jun 16, 2022"
3,Advanced Java: Threads,https://www.linkedin.com/learning/advanced-jav...,1h 14m,By: Buddhini Samarakkody,0,"Released Jun 16, 2022"
4,Understanding Viral Videos,https://www.linkedin.com/learning/understandin...,1h 2m,By: Ashley Kennedy,0,"Released Jun 14, 2022"
5,Developing Your LinkedIn Creator Brand,https://www.linkedin.com/learning/developing-y...,36m,By: Alexandra Galviz,0,"Released Jun 14, 2022"
6,Cisco Networking Foundations: Fundamentals of ...,https://www.linkedin.com/learning/cisco-networ...,1h 50m,By: Kevin Wallace,0,"Released Jun 14, 2022"
7,Autodesk Civil 3D 2023 Essential Training,https://www.linkedin.com/learning/autodesk-civ...,10h 14m,By: Eric Chappell,0,"Released Jun 14, 2022"
8,Google Data Studio for Beginners,https://www.linkedin.com/learning/google-data-...,41m,By: Joshua Rischin,0,"Released Jun 14, 2022"
9,The CEO Test: Mastering Leadership Challenges ...,https://www.linkedin.com/learning/the-ceo-test...,13m,By: Next Big Idea Club,0,"Released Jun 14, 2022"


In [14]:
likesList = []
skillLevelList = []
ratingList = []
ratingMaxList = []

for i in df['url']:
    req = requests.get(i).text
    soup = BeautifulSoup(req)
    temp = soup.find_all('span',{'class':'top-card__headline-row-item'})
    likes = None
    skill = None
    for i in temp:
        if "Liked" in i.text:
            likes = i.text
        if "Skill" in i.text:
            skill = i.text
    likesList.append(likes)
    skillLevelList.append(skill)
    rating = soup.find('span',{'class':'ratings-summary__rating-average'})
    if rating is not None:
        ratingList.append(rating.text)
    else:
        ratingList.append(None)
    ratingMax = soup.find('span',{'class':'ratings-summary__rating-max'})
    if rating is not None:
        ratingMaxList.append(ratingMax.text)
    else:
        ratingMaxList.append(None)

In [15]:
tempdf = pd.DataFrame(list(zip(likesList,skillLevelList,ratingList,ratingMaxList)),
                     columns = ['likes','skill level','rating','rating max'])


df = pd.concat([df,tempdf],axis = 1)
df

,course name,url,duration,by,viewer count,release date,likes,skill level,rating,rating max
0,The Extended Mind: Thinking outside the Brain ...,https://www.linkedin.com/learning/the-extended...,14m,By: Next Big Idea Club,0,"Released Jun 15, 2022",Liked by 1 user,Skill level: Beginner,None,None
1,Branding with Creative Cloud Express,https://www.linkedin.com/learning/branding-wit...,1h 19m,By: Nicte Cuevas,0,"Released Jun 16, 2022",None,Skill level: General,None,None
2,Conducting Research in a Post-Pandemic World,https://www.linkedin.com/learning/conducting-r...,35m,By: Cory Lebson,0,"Released Jun 16, 2022",None,Skill level: Beginner + Intermediate,None,None
3,Advanced Java: Threads,https://www.linkedin.com/learning/advanced-jav...,1h 14m,By: Buddhini Samarakkody,0,"Released Jun 16, 2022",None,Skill level: Advanced,None,None
4,Understanding Viral Videos,https://www.linkedin.com/learning/understandin...,1h 2m,By: Ashley Kennedy,0,"Released Jun 14, 2022",Liked by 4 users,Skill level: General,None,None
5,Developing Your LinkedIn Creator Brand,https://www.linkedin.com/learning/developing-y...,36m,By: Alexandra Galviz,0,"Released Jun 14, 2022",Liked by 4 users,Skill level: Beginner + Intermediate,None,None
6,Cisco Networking Foundations: Fundamentals of ...,https://www.linkedin.com/learning/cisco-networ...,1h 50m,By: Kevin Wallace,0,"Released Jun 14, 2022",Liked by 1 user,Skill level: Beginner,None,None
7,Autodesk Civil 3D 2023 Essential Training,https://www.linkedin.com/learning/autodesk-civ...,10h 14m,By: Eric Chappell,0,"Released Jun 14, 2022",None,Skill level: Beginner,None,None
8,Google Data Studio for Beginners,https://www.linkedin.com/learning/google-data-...,41m,By: Joshua Rischin,0,"Released Jun 14, 2022",Liked by 1 user,Skill level: Beginner,None,None
9,The CEO Test: Mastering Leadership Challenges ...,https://www.linkedin.com/learning/the-ceo-test...,13m,By: Next Big Idea Club,0,"Released Jun 14, 2022",None,Skill level: Beginner,None,None


In [16]:
os.chdir(r'/kaggle/working')

df.to_csv(r'webScrapexp.csv', index = False)